In [1]:
from qkeras import *
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Softmax, Add, ZeroPadding2D, MaxPooling2D
import numpy as np
from collections import namedtuple
import pickle
import math
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from qkeras.utils import model_save_quantized_weights

from bundle import Bundle

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_data(num_classes=10, subtract_pixel_mean=True):
    """
    Load CIFAR10 data and normalize
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # input image dimensions.
    input_shape = x_train.shape[1:]

    # normalize data.
    x_train = x_train.astype('float32') / 128.0 - 1.0
    x_test = x_test.astype('float32') / 128.0 - 1.0

    # if subtract pixel mean is enabled
    if subtract_pixel_mean:
        x_train_mean = np.mean(x_train, axis=0)
        x_train -= x_train_mean
        x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    # convert class vectors to binary class matrices,
    # i.e., one hot encodings
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test


In [3]:
x_train, y_train, x_test, y_test = load_data(10, False)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
input_shape = x_train.shape[1:-1] + (3,)
np.random.seed(1)

a_0 = 'quantized_relu(8,0,negative_slope=0.125)'
a_1 = 'quantized_relu(8,1,negative_slope=0.125)'
a_2 = 'quantized_relu(8,2,negative_slope=0.125)'
a_3 = 'quantized_relu(8,3,negative_slope=0.125)'

q_0 = 'quantized_bits(8,0,False,True,1)'
q_1 = 'quantized_bits(8,1,False,True,1)'
q_2 = 'quantized_bits(8,2,False,True,1)'
q_3 = 'quantized_bits(8,3,False,True,1)'

q_t = 'quantized_bits(8,0,False,True,1)'

np.random.seed(42)
#preamble = './drive/MyDrive/resnet/'
preamble = ''
USE_BIAS = True

In [5]:

'''
Build Model
'''

x = x_in = Input(input_shape, name='input')
x = QActivation(q_0)(x)

x = x1 = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(7,7), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_0},
    pool= {'type':'max', 'size':(3,3), 'strides':(1,1), 'padding':'same', 'act_str': q_0}
    )(x)

# block 0
x = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_0}
    )(x)


x = x1 = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_1},
    add= {'act_str': a_0}
    )(x, x1)

x = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_0}
    )(x)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_1}, 
    add= {'act_str': a_1}
    )(x, x1)

# block 1
x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_1}
    )(x1)
x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2}
    )(x1)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2},
    add={'act_str':a_2}
    )(x, x1)

x = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_1}
    )(x)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2},
    add={'act_str':a_2}
    )(x, x1)

#block 2
x1 = Bundle(
    core= {'type':'conv', 'filters':256, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_1}
    )(x1)
x1 = Bundle(
    core= {'type':'conv', 'filters':256, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2}
    )(x1)
x = x1 = Bundle(
    core = {'type':'conv', 'filters':256, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3},
    add= {'act_str':a_3}
    )(x, x1)

x = Bundle(
    core = {'type':'conv', 'filters':256, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_2}
    )(x)
x = x1 = Bundle(
    core = {'type':'conv', 'filters':256, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3},
    add= {'act_str':a_3}
    )(x, x1)

#block 3
x1 = Bundle(
    core={'type':'conv', 'filters':512, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_2}
    )(x1)
x1 = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3}
    )(x1)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3}, 
    add= {'act_str':a_3}
    )(x, x1)

x = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_2}
    )(x)
x = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3}, 
    add= {'act_str':a_3},
    pool= {'type':'avg', 'size':(2,2), 'strides':(2,2), 'padding':'valid', 'act_str': q_3},
    flatten=True
    )(x, x1)

x = Bundle(
    core= {'type':'dense', 'units':10, 'kernel_quantizer':q_2, 'bias_quantizer':q_2, 'use_bias':USE_BIAS, 'act_str': q_3}, 
    softmax=True)(x)

model = Model(inputs=x_in, outputs=x)
print(model.summary(expand_nested=True))


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 q_activation (QActivation)     (None, 32, 32, 3)    0           ['input[0][0]']                  
                                                                                                  
 bundle (Bundle)                (None, 16, 16, 64)   9729        ['q_activation[0][0]']           
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| q_activation_1 (QActivation)  multiple            0           []                               |
|                                                                                             

In [6]:
def lr_schedule(epoch):
    """
  Bundles_pre_trainearning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    # initial_lr = 1e-4
    # lr_decay = 0.99
    # lr = initial_lr * (lr_decay ** epoch)
    lr = 1e-3 # default 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-2
    elif epoch > 100:
        lr *= 1e-1
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

preamble = ''
model_file_path = preamble+'resnet18.h5'
checkpoint = ModelCheckpoint(filepath=model_file_path,
                                     monitor='val_acc',
                                     verbose=1,
                                     save_best_only=True)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                       cooldown=0,
                                       patience=5,
                                       min_lr=0.5e-6)
lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

NB_EPOCH = 200
BATCH_SIZE = 256
VERBOSE = 1
VALIDATION_SPLIT = 0.1
RELU_NEG_SLOPE = 0.125

model.compile(loss='categorical_crossentropy',
            optimizer=Adam(learning_rate=lr_schedule(0)), metrics=['acc'])

# model.fit(x_train, y_train,
#                     batch_size=BATCH_SIZE,
#                     epochs=NB_EPOCH,
#                     validation_data=(x_test, y_test),
#                     shuffle=True,
#                     callbacks=callbacks)

Learning rate:  0.001


In [7]:
XN = 4
x = np.random.randn(XN, *model.input.shape[1:])
x = np.clip(x, -1.0, 1.0)

inp_act_model = Model(inputs=model.input, outputs=model.layers[1].output)
inp ={ 'tensor': inp_act_model(x, training=False), 'bits':8, 'frac':7}
inp['int'] = inp['tensor'].numpy() * 2**inp['frac']

y = model(x)

model.layers[2].process(inp)
for layer in model.layers[3:]:
    layer.process()



In [8]:
with open('../compile.pickle', 'rb') as f:
    compile_d = pickle.load(f)
    c = namedtuple('Compile', compile_d)(**compile_d)

bundles = model.layers[2:]
for bundle in bundles:
    print(f'-----------------{bundle.idx}-----------------------')
    bundle.export(c)



-----------------0-----------------------
weights initial (KH, KW, CI, CO) = (7, 7, 3, 64)
KH=7, KW=7, CI=3, CO=64, CO_PRL=3, EG=3, IT=22, 66
input initial (XN, XH, XW, CI)= (4, 32, 32, 3)
output initial (4, 32, 32, 64)
{'w_shape': (7, 7, 3, 64), 'x_shape': (4, 32, 32, 3), 'y_shape': (4, 32, 32, 64), 'SW': 1, 'SH': 1, 'KH': 7, 'KW': 7, 'CI': 3, 'CO': 64, 'CO_PRL': 3, 'EG': 3, 'IT': 22, 'CO_PAD': 66, 'XN': 4, 'XH': 32, 'XW': 32, 'SH_OUT': 1, 'SW_OUT': 1, 'LH': 8, 'L': 4, 'XH_PAD': 32, 'BRAM_WEIGHTS_ADDR_MAX': 21}
Runtime(w_shape=(7, 7, 3, 64), x_shape=(4, 32, 32, 3), y_shape=(4, 32, 32, 64), SW=1, SH=1, KH=7, KW=7, CI=3, CO=64, CO_PRL=3, EG=3, IT=22, CO_PAD=66, XN=4, XH=32, XW=32, SH_OUT=1, SW_OUT=1, LH=8, L=4, XH_PAD=32, BRAM_WEIGHTS_ADDR_MAX=21, w_config='0b00000000000000000000000000000000101010011111111100000000010011', w_config_words=array([[ 19, -64, 127,  42,   0,   0,   0,   0],
       [ 19, -64, 127,  42,   0,   0,   0,   0],
       [ 19, -64, 127,  42,   0,   0,   0,   0],
    